# Calculating Coverage Percentage of the AOI by an Item

This notebook demonstrates the use of pystac-client to calculate the percentage an Item's geometry that intesects with the area of interest (AOI) specified in the search by the `intersects` parameter.

In [ ]:
from pystac_client import Client

import matplotlib.pyplot as plt
from shapely.geometry import shape
from pystac_client import Client
from pystac.item import Item
from typing import Dict, Any

In [ ]:
def intersection_percent(item: Item, aoi: Dict[str, Any]) -> float:
    '''The percentage that the Item's geometry intersects the AOI. An Item that
    completely covers the AOI has a value of 100.
    '''
    geom_item = shape(item.geometry)
    geom_aoi = shape(aoi)

    intersected_geom = geom_aoi.intersection(geom_item)

    intersection_percent = (intersected_geom.area * 100) / geom_aoi.area

    return intersection_percent


# STAC API root URL
URL = 'https://planetarycomputer.microsoft.com/api/stac/v1'

# geometry of the AOI to search over
intersects_geometry = {
    "type": "Polygon",
    "coordinates": [
        [
            [-73.21, 43.99],
            [-73.21, 47.05],
            [-70.12, 47.05],
            [-70.12, 43.99],
            [-73.21, 43.99],
        ]
    ],
}

# Create a Client and an ItemSearch representing our search
# No search operations will be performed until we call the items() method
client = Client.open(URL, ignore_conformance=False)
item_search = client.search(
    collections=["sentinel-2-l2a"], intersects=intersects_geometry, max_items=100
)

In [ ]:
print([f"{intersection_percent(item, intersects_geometry):.2f}" for item in item_search.items()])

In [ ]:
# create a generator that filters to only those Items that intersect more than 5%
items_gt_5_percent = (
    i for i in item_search.items() if intersection_percent(i, intersects_geometry) > 5
)

In [ ]:
# Render the AOI and Item results
# The green shape is the AOI
# The blue shapes are the Item geometries
# If there are no blue shapes, adjust the intersection percent filter above until there are

cm = plt.get_cmap('RdBu')
fig, axs = plt.subplots()
axs.set_aspect('equal', 'datalim')

for item in items_gt_5_percent:
  xs, ys = shape(item.geometry).exterior.xy
  axs.fill(xs, ys, alpha=0.5, fc='b', ec='none')

geom_intersects = shape(intersects_geometry)
xs, ys = geom_intersects.exterior.xy
axs.fill(xs, ys, alpha=0.5, fc='g', ec='none')

plt.show()